In [1]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
import torch.nn.functional as F
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [2]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target

In [3]:
X = X[(y == '1') | (y == '3') | (y == '5') | (y == '7')]
y = y[(y == '1') | (y == '3') | (y == '5') | (y == '7')]

map = {'1': 0, '3': 1, '5': 2, '7': 3}
y = y.map(map)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)

scaler = MinMaxScaler(feature_range=(0, np.pi/2))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [5]:
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test_torch  = torch.tensor(X_test,  dtype=torch.float32)
y_test_torch  = torch.tensor(y_test.to_numpy(),  dtype=torch.long)

train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset  = TensorDataset(X_test_torch,  y_test_torch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

In [6]:
def sample_from_train(X_train, y_train, sample_size_for_each_class, batch_size):
    X_train_sampled = []
    y_train_sampled = []

    y_train_np = y_train.to_numpy()

    for class_label in np.unique(y_train_np):
        class_indices = np.where(y_train_np == class_label)[0]
        sampled_indices = np.random.choice(class_indices, size=sample_size_for_each_class, replace=False)

        X_train_sampled.extend(X_train[sampled_indices])
        y_train_sampled.extend(y_train_np[sampled_indices])

    X_train_sampled = np.array(X_train_sampled)
    y_train_sampled = np.array(y_train_sampled)

    X_train_sampled_torch = torch.tensor(X_train_sampled, dtype=torch.float32)
    y_train_sampled_torch = torch.tensor(y_train_sampled, dtype=torch.long)

    train_sampled_dataset = TensorDataset(X_train_sampled_torch, y_train_sampled_torch)
    train_sampled_loader = DataLoader(train_sampled_dataset, batch_size=batch_size, shuffle=True)

    return train_sampled_loader

In [7]:
class QuantumLayer():
    def __init__(self, n_qubits, n_layers):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.dev = qml.device('lightning.gpu', wires=self.n_qubits)

    def qlayer(self, inputs, weights):
        @qml.qnode(self.dev, interface="torch")
        def circuit(inputs, weights):
            qml.AngleEmbedding(inputs, wires=list(np.arange(self.n_qubits)), rotation='X')
            qml.StronglyEntanglingLayers(weights, wires=range(self.n_qubits))
            return [qml.expval(qml.PauliZ(i) @ qml.PauliZ((i+1)%self.n_qubits)) for i in range(self.n_qubits)] 

        return circuit(inputs, weights)  


class HybridClassifier(nn.Module):
    def __init__(self, n_qubits, n_layers, input_dim, learning_rate=1e-3, batch_size=32, epochs=3):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.quantum_layer = QuantumLayer(n_qubits, n_layers)

        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 8)
        self.scale = nn.Parameter(torch.tensor([2 * np.pi]))
        self.qnn_weights = nn.Parameter(torch.rand(n_layers, n_qubits, 3) * np.pi)
        self.output_layer = nn.Linear(n_qubits, 4)

        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.to(self.device)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = torch.tanh(x) * self.scale

        batch_size = x.size(0)
        out = torch.empty((batch_size, self.n_qubits), dtype=torch.float32, device=x.device)

        for batch_idx in range(batch_size):
            expvals = self.quantum_layer.qlayer(x[batch_idx], self.qnn_weights)
            out[batch_idx] = torch.stack(expvals)

        x = self.output_layer(out)
        return x

    def train_model(self, dataloader, loss_fn, optimizer):
        self.train()
        for _, (X, y) in enumerate(dataloader):
            X, y = X.to(self.device), y.to(self.device)
            pred = self(X)
            loss = loss_fn(pred, y)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    def test_model(self, dataloader, loss_fn):
        self.eval()
        num_batches = len(dataloader)
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(self.device), y.to(self.device)
                pred = self(X)
                predicted_label = torch.argmax(pred, dim=1)
                correct += torch.count_nonzero(predicted_label == y)
                test_loss += loss_fn(pred, y).item()

        accuracy = (correct / (len(dataloader) * self.batch_size)) * 100
        test_loss /= num_batches
        print(f"Accuracy: {accuracy:.2f}%")
        print(f"Avg loss: {test_loss:>8f} \n")

        return test_loss

    def train_with_scheduler(self, train_loader):
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.parameters(), lr=self.learning_rate, momentum=0.9, nesterov=True)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

        train_indices = np.arange(len(train_loader.dataset))
        np.random.shuffle(train_indices)
        val_size = int(0.1 * len(train_indices))  
        val_indices, train_indices = train_indices[:val_size], train_indices[val_size:]

        val_subset = Subset(train_loader.dataset, val_indices)
        val_loader = DataLoader(val_subset, batch_size=self.batch_size, shuffle=False)

        start_time = time.time()
 
        for epoch in range(self.epochs):
            print(f"Epoch {epoch+1}\n--------------------------")
            self.train_model(train_loader, loss_fn, optimizer)
            val_loss = self.test_model(val_loader, loss_fn)  
            scheduler.step(val_loss)

        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Training took {elapsed_time/60:.2f} minutes.")

In [ ]:
class ClassicalClassifier(nn.Module):
    def __init__(self, input_dim, learning_rate=1e-3, batch_size=32, epochs=20):
        super().__init__()
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 8)
        self.output_layer = nn.Linear(8, 4)  

        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.to(self.device)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output_layer(x)
        return x

    def train_model(self, dataloader, loss_fn, optimizer):
        self.train()
        for _, (X, y) in enumerate(dataloader):
            X, y = X.to(self.device), y.to(self.device)
            pred = self(X)
            loss = loss_fn(pred, y)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    def test_model(self, dataloader, loss_fn):
        self.eval()
        num_batches = len(dataloader)
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(self.device), y.to(self.device)
                pred = self(X)
                predicted_label = torch.argmax(pred, dim=1)
                correct += torch.count_nonzero(predicted_label == y)
                test_loss += loss_fn(pred, y).item()

        accuracy = (correct / (len(dataloader) * self.batch_size)) * 100
        test_loss /= num_batches
        print(f"Accuracy: {accuracy:.2f}%")
        print(f"Avg loss: {test_loss:>8f} \n")

        return test_loss, accuracy

    def train_with_scheduler(self, train_loader):
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(self.parameters(), lr=self.learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

        train_indices = np.arange(len(train_loader.dataset))
        np.random.shuffle(train_indices)
        val_size = int(0.1 * len(train_indices))  
        val_indices, train_indices = train_indices[:val_size], train_indices[val_size:]

        val_subset = Subset(train_loader.dataset, val_indices)
        val_loader = DataLoader(val_subset, batch_size=self.batch_size, shuffle=False)

        start_time = time.time()
 
        for epoch in range(self.epochs):
            print(f"Epoch {epoch+1}\n--------------------------")
            self.train_model(train_loader, loss_fn, optimizer)
            val_loss = self.test_model(val_loader, loss_fn)  
            scheduler.step(val_loss)

        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Training took {elapsed_time/60:.2f} minutes.")

In [ ]:
sampled_train_loader = sample_from_train(X_train, y_train, sample_size_for_each_class=20, batch_size=32)

In [15]:
n_qubits = 8
n_layers = 4
input_dim = X_train.shape[1]

model = HybridClassifier(n_qubits, n_layers, input_dim, batch_size=32, epochs=20, learning_rate=0.01)

print("\nTraining with sampled dataset:")
model.train_with_scheduler(sampled_train_loader)

print("\nEvaluating on test set:")
model.test_model(test_loader, loss_fn=nn.CrossEntropyLoss())

LightningException: [/project/pennylane_lightning/core/src/utils/cuda_utils/DevicePool.hpp][Line:86][Method:getTotalDevices]: Error in PennyLane Lightning: unknown error

In [10]:
input_dim = X_train.shape[1]

model_classical = ClassicalClassifier(input_dim, batch_size=32, epochs=30, learning_rate=0.01)


print("Training Classical Model with sample Dataset:")
model_classical.train_with_scheduler(sampled_train_loader)

print("\nEvaluating on test set:")
model_classical.test_model(test_loader, loss_fn=nn.CrossEntropyLoss())

/home/abdelrahmanelsayed/PC/Quantum_Abdo/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Training Classical Model with sample Dataset:
Epoch 1
--------------------------
Accuracy: 21.88%
Avg loss: 1.122073 

Epoch 2
--------------------------
Accuracy: 21.88%
Avg loss: 0.883775 

Epoch 3
--------------------------
Accuracy: 25.00%
Avg loss: 0.531198 

Epoch 4
--------------------------
Accuracy: 18.75%
Avg loss: 0.251568 

Epoch 5
--------------------------
Accuracy: 25.00%
Avg loss: 0.142732 

Epoch 6
--------------------------
Accuracy: 25.00%
Avg loss: 0.003685 

Epoch 7
--------------------------
Accuracy: 25.00%
Avg loss: 0.000110 

Epoch 8
--------------------------
Accuracy: 25.00%
Avg loss: 0.000579 

Epoch 9
--------------------------
Accuracy: 25.00%
Avg loss: 0.000634 

Epoch 10
--------------------------
Accuracy: 25.00%
Avg loss: 0.002928 

Epoch 11
--------------------------
Accuracy: 25.00%
Avg loss: 0.045063 

Epoch 12
--------------------------
Accuracy: 25.00%
Avg loss: 0.004634 

Epoch 13
--------------------------
Accuracy: 25.00%
Avg loss: 0.003338 

E

1.0265402381467268

In [11]:
input_dim = X_train.shape[1]

model_classical = ClassicalClassifier(input_dim, batch_size=32, epochs=30, learning_rate=0.01)


print("Training Classical Model with Full Dataset:")
model_classical.train_with_scheduler(train_loader)

print("\nEvaluating on test set:")
model_classical.test_model(test_loader, loss_fn=nn.CrossEntropyLoss())

Training Classical Model with Full Dataset:
Epoch 1
--------------------------
Accuracy: 93.12%
Avg loss: 0.244655 

Epoch 2
--------------------------
Accuracy: 98.44%
Avg loss: 0.054106 

Epoch 3
--------------------------
Accuracy: 98.75%
Avg loss: 0.040742 

Epoch 4
--------------------------
Accuracy: 98.96%
Avg loss: 0.044608 

Epoch 5
--------------------------
Accuracy: 98.59%
Avg loss: 0.048365 

Epoch 6
--------------------------
Accuracy: 99.06%
Avg loss: 0.029403 

Epoch 7
--------------------------
Accuracy: 98.65%
Avg loss: 0.036461 

Epoch 8
--------------------------
Accuracy: 99.22%
Avg loss: 0.020570 

Epoch 9
--------------------------
Accuracy: 98.39%
Avg loss: 0.053110 

Epoch 10
--------------------------
Accuracy: 98.91%
Avg loss: 0.096479 

Epoch 11
--------------------------
Accuracy: 99.48%
Avg loss: 0.014507 

Epoch 12
--------------------------
Accuracy: 99.48%
Avg loss: 0.023573 

Epoch 13
--------------------------
Accuracy: 97.03%
Avg loss: 0.166527 

Epo

0.13154431012679194